# Example workflow

In [2]:
import rectanglepy as rectangle
from anndata import AnnData

## Load tutorial data

In [5]:
sc_counts, annotations, bulks  = rectangle.load_tutorial_data()

In [24]:
sc_counts.iloc[:, :5].head()

,MIR1302-2HG,AL627309.1,AL627309.4,AC114498.1,AL669831.5
E2L4_GATGCTACAGGCACAA,0,0,0,0,0
L5_AACAACCAGGAACTAT,0,0,0,0,0
L5_TCCTTCTGTACTCCGG,0,0,0,0,0
L2_GCCCGAACACGTATAC,0,0,0,0,0
E2L2_ATGCATGTCACACCCT,0,0,0,0,0


In [25]:
annotations.head()

E2L4_GATGCTACAGGCACAA    Monocytes
L5_AACAACCAGGAACTAT      Monocytes
L5_TCCTTCTGTACTCCGG      Monocytes
L2_GCCCGAACACGTATAC      Monocytes
E2L2_ATGCATGTCACACCCT    Monocytes
Name: 0, dtype: object

In [26]:
bulks.iloc[:, :5].head()

,UBE2Q2P2,SSX9,CXorf67,EFCAB8,SPATA31B1P
pbmc_1,0.000000,0.0,0.118865,0.00000,0.0
pbmc_10,0.081115,0.0,0.086782,0.00000,0.0
pbmc_12,0.000000,0.0,0.188464,0.03157,0.0


### Convert to AnnData object

In [6]:
sc_adata = AnnData(sc_counts, obs=annotations.to_frame(name="cell_type"))

# Single step Rectangle workflow

To deconvolute the bulk data in a single step, use the rectangle function. This function returns a tuple of the estimated cell type proportions and the signature result.

In [ ]:
estimations, signature_result = rectangle.rectangle(sc_adata, bulks)

In [ ]:
estimations

signature_result is a [`RectangleSignatureResult`](../generated/rectanglepy.pp.RectangleSignatureResult.rst) object.

# 2-step Rectangle workflow

Rectangle can also be run in two steps, first creating the signature and then deconvoluting the bulk data.

## Create Signature result

In [ ]:
signature_result = rectangle.pp.build_rectangle_signatures(sc_adata, bulks=bulks)

This creates a [`RectangleSignatureResult`](../generated/rectanglepy.pp.RectangleSignatureResult.rst) object.

## Deconvolute bulk data

In [ ]:
estimations = rectangle.tl.deconvolution(signature_result, bulks)

This returns a pandas DataFrame with the estimated cell type proportions.

In [14]:
estimations.T

,pbmc_1,pbmc_10,pbmc_12
B cells,0.089947,1.099018e-01,7.147636e-02
ILC,0.008037,1.482882e-02,7.568929e-03
Monocytes,0.227473,2.967478e-01,2.255510e-01
NK cells,0.022183,1.626444e-02,1.011593e-01
Plasma cells,0.005894,6.567050e-04,3.095959e-03
Platelet,0.004847,2.190904e-02,1.863999e-02
T cells CD4 conv,0.037192,-6.450194e-18,-3.137140e-17
T cells CD8,0.161779,1.180941e-01,2.740643e-01
Tregs,0.416100,3.962703e-01,2.563935e-01
mDC,0.024173,2.066734e-02,1.005506e-02


# Analysis of unknown content estimations

Rectangle not only gives you an estimation of unkown content, it also gives you gene wise estimates of the unknown content.

The correlation between the unknown cell type and the genes linked to the unknown cell type. There are two columns:
1. `corr_expr`: the correlation between the unknown cell type and the expression of the genes in the bulk.
2. `corr_err`: the correlation between the unknown cell type and the error (bulk - bulk_est) calculated during the bulk reconstruction step of the unknown content.

In [8]:
unkn_gene_corr = signature_result.unkn_gene_corr
unkn_gene_corr.head()

,corr_expr,corr_err
A1BG-AS1,-0.470345,-0.433278
A2M,0.203560,-0.797782
AAAS,-0.957780,-0.980177
AAED1,0.999920,0.999998
ABAT,-0.727324,-0.144902


And also the genewise result of ‘bulk - bulk_est’, which is the difference between the  expression in the bulk and the estimated expression in the reconstructed used to calculate the unknown content:

In [21]:
unkn_bulk_err = signature_result.unkn_bulk_err
unkn_bulk_err.iloc[:, :5].head()

,A1BG-AS1,A2M,AAAS,AAED1,ABAT
pbmc_1,1.757772,-0.551459,10.402414,-8.103443,-8.706732
pbmc_10,0.700000,-0.729346,9.421015,-8.095237,-11.249540
pbmc_12,0.788494,-0.844238,5.706905,-4.165561,-10.300633
